# Getting Started

In [1]:
%reload_ext autoreload
%autoreload 2
from recsys.MovieLens import MovieLens
from surprise import SVD

In [2]:
def create_anti_testset(user_id, trainset):
    fill = trainset.global_mean
    anti_testset = []
    inner_uid = trainset.to_inner_uid(str(user_id))
    
    user_items = set([j for (j, _) in trainset.ur[inner_uid]])
    anti_testset += [(trainset.to_raw_uid(inner_uid), trainset.to_raw_iid(i), fill) for
                             i in trainset.all_items() if
                             i not in user_items]
    return anti_testset

In [3]:
# Pick an arbitrary test subject
test_uid = 85

lens, ratings_data, rankings = MovieLens.load()

user_ratings = lens.get_user_ratings(test_uid)
loved = []
hated = []
for ratings in user_ratings:
    if (float(ratings[1]) > 4.0):
        loved.append(ratings)
    if (float(ratings[1]) < 3.0):
        hated.append(ratings)

loved.sort(key=lambda x: x[1], reverse=True)
hated.sort(key=lambda x: x[1])

print(f"User {test_uid} loved these movies:")
for ratings in loved[:10]:
    print(lens.get_movie_name(ratings[0]), ratings[1])
print("\n...and hated these movies:")
for ratings in hated[:10]:
    print(lens.get_movie_name(ratings[0]), ratings[1])

User 85 loved these movies:
Jumanji (1995) 5.0
GoldenEye (1995) 5.0
Braveheart (1995) 5.0
Jerky Boys, The (1995) 5.0
LÃ©on: The Professional (a.k.a. The Professional) (LÃ©on) (1994) 5.0
Pulp Fiction (1994) 5.0
Stargate (1994) 5.0
Shawshank Redemption, The (1994) 5.0
Star Trek: Generations (1994) 5.0
Clear and Present Danger (1994) 5.0

...and hated these movies:
Postman, The (Postino, Il) (1994) 1.0
Casper (1995) 1.0
Mighty Morphin Power Rangers: The Movie (1995) 1.0
Prophecy, The (1995) 1.0
Heavenly Creatures (1994) 1.0
Little Women (1994) 1.0
Nell (1994) 1.0
Poison Ivy II (1996) 1.0
Tank Girl (1995) 1.0
While You Were Sleeping (1995) 1.0


In [4]:
# Building recommendation model
trainset = ratings_data.build_full_trainset()

model = SVD(verbose=False)
model.fit(trainset)

In [5]:
# Generate recommendations
testset = create_anti_testset(test_uid, trainset)
predictions = model.test(testset)

recommendations = []

ratings_data = lens.load_movielens_data()

print ("We recommend:\n")
for user_id, movie_id, true_rating, predicted_rating, _ in predictions:
    recommendations.append((int(movie_id), predicted_rating))

recommendations.sort(key=lambda x: x[1], reverse=True)

for ratings in recommendations[:10]:
    print(lens.get_movie_name(ratings[0]), f"{ratings[1]:.1f}")

We recommend:

Godfather: Part II, The (1974) 4.5
Lord of the Rings: The Return of the King, The (2003) 4.4
Fight Club (1999) 4.3
Conversation, The (1974) 4.3
Psycho (1960) 4.3
All the President's Men (1976) 4.3
Lawrence of Arabia (1962) 4.3
Usual Suspects, The (1995) 4.3
Lord of the Rings: The Fellowship of the Ring, The (2001) 4.3
Producers, The (1968) 4.3
